-  DSA dataset을 이용해서, 정상데이터(Lying)과 비정상데이터 설정 (다른 jumping이 아니라 activity 선택)

- Encoder와 Decoder의 구조가 같은 오토인코더로 지난주차와 동일하게 이상치 탐지

- Encoder와 Decoder의 구조가 다른 비대칭형 뉴런수로 성능 확인

- DAE를 사용하여 성능확인

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras import layers, models
import tensorflow as tf


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2024-11-21 00:25:47.799141: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv("datas/DSA_features.csv")
df

,T_xacc_mean,T_xacc_max,T_xacc_min,T_xacc_var,T_xacc_std,T_xacc_skew,T_yacc_mean,T_yacc_max,T_yacc_min,T_yacc_var,...,LL_ymag_std,LL_ymag_skew,LL_zmag_mean,LL_zmag_max,LL_zmag_min,LL_zmag_var,LL_zmag_std,LL_zmag_skew,activity,people
0,7.975714,8.1605,7.6823,0.014395,0.119981,-0.023319,1.083150,1.1832,0.99744,0.002208,...,0.000792,0.177075,-0.057119,-0.054963,-0.059241,6.778722e-07,0.000823,0.036729,sitting,p1
1,7.978250,8.1763,7.8472,0.007551,0.086896,0.552416,1.140865,1.2129,1.05810,0.000784,...,0.000860,-0.286918,-0.057268,-0.054945,-0.059589,7.032302e-07,0.000839,0.347471,sitting,p1
2,7.970894,8.0860,7.8470,0.003092,0.055603,0.100538,1.140962,1.2128,1.07960,0.000508,...,0.000762,-0.134430,-0.057068,-0.054711,-0.059065,6.268222e-07,0.000792,0.045579,sitting,p1
3,7.938412,8.1083,7.6901,0.003763,0.061343,-0.231914,1.165260,1.3170,1.07870,0.002173,...,0.000735,0.021485,-0.056422,-0.053670,-0.058310,8.011245e-07,0.000895,0.240690,sitting,p1
4,7.908930,8.1305,7.8322,0.001741,0.041731,2.042285,1.187504,1.2574,1.09450,0.000662,...,0.000824,-0.148229,-0.055801,-0.053313,-0.057815,6.853423e-07,0.000828,0.258429,sitting,p1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9115,8.280854,34.1980,-2.9038,28.080803,5.299132,1.350075,-1.491537,11.2240,-11.65100,14.670334,...,0.200829,-0.040701,0.297666,0.708480,-0.117430,4.135451e-02,0.203358,-0.310022,basketBall,p8
9116,9.591118,51.6970,-3.4129,35.722025,5.976791,2.981144,0.086304,6.9951,-11.76400,5.329897,...,0.148745,-0.266377,0.224716,0.554670,-0.250950,3.355704e-02,0.183186,-0.736410,basketBall,p8
9117,9.599113,27.9300,-1.0765,48.850886,6.989341,0.449237,-0.728367,3.7801,-8.36910,5.683022,...,0.310748,-0.009505,-0.237786,0.088854,-0.477260,2.026107e-02,0.142341,0.668438,basketBall,p8
9118,9.692482,72.7820,-2.6734,59.378336,7.705734,4.491114,-0.582724,6.1216,-8.85710,4.162963,...,0.156493,0.050624,0.533023,0.677800,0.055941,1.356379e-02,0.116464,-1.482489,basketBall,p8


In [5]:
normal = df[(df['activity'] == 'lyingRigh') | (df['activity'] == 'lyingBack')]
abnormal = df[df['activity'] == 'jumping']

X_normal = normal.drop(columns=['activity']).select_dtypes(include=[np.number])
X_abnormal = abnormal.drop(columns=['activity']).select_dtypes(include=[np.number])


In [6]:
scaler = StandardScaler()
X_normal_scaled = scaler.fit_transform(X_normal)
X_abnormal_scaled = scaler.transform(X_abnormal)

def build_symmetric_autoencoder(input_dim):
    input_layer = layers.Input(shape=(input_dim,))
    encoded = layers.Dense(64, activation='relu')(input_layer)
    decoded = layers.Dense(input_dim, activation='sigmoid')(encoded)
    
    autoencoder = models.Model(input_layer, decoded)
    autoencoder.compile(optimizer='adam', loss='mean_squared_error')
    
    return autoencoder

autoencoder_symmetric = build_symmetric_autoencoder(X_normal_scaled.shape[1])
autoencoder_symmetric.fit(X_normal_scaled, X_normal_scaled, epochs=50, batch_size=256, validation_split=0.2)


2024-11-21 00:28:10.218722: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/50
3/3 [==============================] - 1s 112ms/step - loss: 1.3070 - val_loss: 0.9952
Epoch 2/50
3/3 [==============================] - 0s 16ms/step - loss: 1.2807 - val_loss: 0.9786
Epoch 3/50
3/3 [==============================] - 0s 19ms/step - loss: 1.2559 - val_loss: 0.9622
Epoch 4/50
3/3 [==============================] - 0s 19ms/step - loss: 1.2306 - val_loss: 0.9455
Epoch 5/50
3/3 [==============================] - 0s 17ms/step - loss: 1.2023 - val_loss: 0.9281
Epoch 6/50
3/3 [==============================] - 0s 24ms/step - loss: 1.1719 - val_loss: 0.9097
Epoch 7/50
3/3 [==============================] - 0s 18ms/step - loss: 1.1391 - val_loss: 0.8907
Epoch 8/50
3/3 [==============================] - 0s 17ms/step - loss: 1.1061 - val_loss: 0.8713
Epoch 9/50
3/3 [==============================] - 0s 18ms/step - loss: 1.0734 - val_loss: 0.8519
Epoch 10/50
3/3 [==============================] - 0s 53ms/step - loss: 1.0430 - val_loss: 0.8333
Epoch 11/50
3/3 [===========

In [7]:
def build_asymmetric_autoencoder(input_dim):
    input_layer = layers.Input(shape=(input_dim,))
    encoded = layers.Dense(128, activation='relu')(input_layer) 
    encoded = layers.Dense(64, activation='relu')(encoded)
    decoded = layers.Dense(128, activation='relu')(encoded) 
    decoded = layers.Dense(input_dim, activation='sigmoid')(decoded)
    
    autoencoder = models.Model(input_layer, decoded)
    autoencoder.compile(optimizer='adam', loss='mean_squared_error')
    
    return autoencoder

autoencoder_asymmetric = build_asymmetric_autoencoder(X_normal_scaled.shape[1])
autoencoder_asymmetric.fit(X_normal_scaled, X_normal_scaled, epochs=50, batch_size=256, validation_split=0.2)


Epoch 1/50
3/3 [==============================] - 1s 98ms/step - loss: 1.3067 - val_loss: 0.9870
Epoch 2/50
3/3 [==============================] - 0s 20ms/step - loss: 1.2819 - val_loss: 0.9662
Epoch 3/50
3/3 [==============================] - 0s 22ms/step - loss: 1.2502 - val_loss: 0.9379
Epoch 4/50
3/3 [==============================] - 0s 52ms/step - loss: 1.2070 - val_loss: 0.9018
Epoch 5/50
3/3 [==============================] - 0s 22ms/step - loss: 1.1521 - val_loss: 0.8614
Epoch 6/50
3/3 [==============================] - 0s 20ms/step - loss: 1.0921 - val_loss: 0.8226
Epoch 7/50
3/3 [==============================] - 0s 19ms/step - loss: 1.0371 - val_loss: 0.7935
Epoch 8/50
3/3 [==============================] - 0s 20ms/step - loss: 0.9968 - val_loss: 0.7744
Epoch 9/50
3/3 [==============================] - 0s 22ms/step - loss: 0.9718 - val_loss: 0.7614
Epoch 10/50
3/3 [==============================] - 0s 25ms/step - loss: 0.9558 - val_loss: 0.7510
Epoch 11/50
3/3 [============

In [8]:

def build_dae_autoencoder(input_dim):
    input_layer = layers.Input(shape=(input_dim,))
    noisy_input = layers.GaussianNoise(0.1)(input_layer) 
    encoded = layers.Dense(64, activation='relu')(noisy_input)
    decoded = layers.Dense(input_dim, activation='sigmoid')(encoded)
    
    autoencoder = models.Model(input_layer, decoded)
    autoencoder.compile(optimizer='adam', loss='mean_squared_error')
    
    return autoencoder

autoencoder_dae = build_dae_autoencoder(X_normal_scaled.shape[1])
autoencoder_dae.fit(X_normal_scaled, X_normal_scaled, epochs=50, batch_size=256, validation_split=0.2)


Epoch 1/50
3/3 [==============================] - 1s 74ms/step - loss: 1.3183 - val_loss: 1.0119
Epoch 2/50
3/3 [==============================] - 0s 50ms/step - loss: 1.2913 - val_loss: 0.9977
Epoch 3/50
3/3 [==============================] - 0s 27ms/step - loss: 1.2669 - val_loss: 0.9831
Epoch 4/50
3/3 [==============================] - 0s 27ms/step - loss: 1.2422 - val_loss: 0.9685
Epoch 5/50
3/3 [==============================] - 0s 56ms/step - loss: 1.2157 - val_loss: 0.9536
Epoch 6/50
3/3 [==============================] - 0s 46ms/step - loss: 1.1868 - val_loss: 0.9382
Epoch 7/50
3/3 [==============================] - 0s 28ms/step - loss: 1.1556 - val_loss: 0.9225
Epoch 8/50
3/3 [==============================] - 0s 24ms/step - loss: 1.1232 - val_loss: 0.9064
Epoch 9/50
3/3 [==============================] - 0s 22ms/step - loss: 1.0905 - val_loss: 0.8900
Epoch 10/50
3/3 [==============================] - 0s 23ms/step - loss: 1.0592 - val_loss: 0.8735
Epoch 11/50
3/3 [============

In [9]:
def calculate_reconstruction_error(autoencoder, X_normal_scaled, X_abnormal_scaled):
    normal_reconstructed = autoencoder.predict(X_normal_scaled)
    abnormal_reconstructed = autoencoder.predict(X_abnormal_scaled)
    
    normal_reconstruction_error = np.mean(np.square(X_normal_scaled - normal_reconstructed), axis=1)
    abnormal_reconstruction_error = np.mean(np.square(X_abnormal_scaled - abnormal_reconstructed), axis=1)
    
    return normal_reconstruction_error, abnormal_reconstruction_error

normal_reconstruction_error_symmetric, abnormal_reconstruction_error_symmetric = calculate_reconstruction_error(autoencoder_symmetric, X_normal_scaled, X_abnormal_scaled)
normal_reconstruction_error_asymmetric, abnormal_reconstruction_error_asymmetric = calculate_reconstruction_error(autoencoder_asymmetric, X_normal_scaled, X_abnormal_scaled)
normal_reconstruction_error_dae, abnormal_reconstruction_error_dae = calculate_reconstruction_error(autoencoder_dae, X_normal_scaled, X_abnormal_scaled)


15/15 [==============================] - 0s 918us/step


In [10]:
threshold_symmetric = np.percentile(normal_reconstruction_error_symmetric, 95)
threshold_asymmetric = np.percentile(normal_reconstruction_error_asymmetric, 95)
threshold_dae = np.percentile(normal_reconstruction_error_dae, 95)


In [11]:
def calculate_accuracy(normal_error, abnormal_error, threshold):
    y_true = np.concatenate([np.zeros(len(normal_error)), np.ones(len(abnormal_error))])
    y_pred = np.concatenate([ (normal_error > threshold).astype(int), (abnormal_error > threshold).astype(int) ])
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy

accuracy_symmetric = calculate_accuracy(normal_reconstruction_error_symmetric, abnormal_reconstruction_error_symmetric, threshold_symmetric)
accuracy_asymmetric = calculate_accuracy(normal_reconstruction_error_asymmetric, abnormal_reconstruction_error_asymmetric, threshold_asymmetric)
accuracy_dae = calculate_accuracy(normal_reconstruction_error_dae, abnormal_reconstruction_error_dae, threshold_dae)

print(f'Accuracy (Symmetric Autoencoder): {accuracy_symmetric * 100:.2f}%')
print(f'Accuracy (Asymmetric Autoencoder): {accuracy_asymmetric * 100:.2f}%')
print(f'Accuracy (Denoising Autoencoder): {accuracy_dae * 100:.2f}%')

Accuracy (Symmetric Autoencoder): 96.67%
Accuracy (Asymmetric Autoencoder): 96.67%
Accuracy (Denoising Autoencoder): 96.67%
